# <font color=#c51b8a>VPOD 'Mine-n-Match':</font>
## <font color=#c994c7>Part 1 Objective</font> - Use Species Names from Microspectrophotemetry Data Sheet to Query NCBI for All Related Opsin Sequences  

In [1]:
import os
import re
import datetime
import time
import pandas as pd
from deepBreaks.preprocessing import read_data
from Bio import Entrez, SeqIO
email = 'sethfrazer@ucsb.edu'

In [2]:
from mnm_scripts.mine_n_match_functions import ncbi_fetch_opsins, get_random_protein_fasta

### <font color=#c994c7>Load data-table with all of the species and sequence data</font> 

In [3]:
species_list = []

In [ ]:
scp_file = './data_sources/vpod/scp_raw.tsv'
scp_df = pd.read_csv(scp_file, index_col=0, sep='\t')
scp_df.head()

In [5]:
species_list+=(scp_df['Genus'] + ' ' + scp_df['Species']).to_list()

In [ ]:
longcore_file = './data_sources/longcore_data/AnimalPhotopigmentsV1_1.csv'
longcore_df = pd.read_csv(longcore_file, index_col=0)
longcore_df.head()

In [7]:
species_list+=longcore_df['Full_Species'].to_list()

In [ ]:
murphy_westerman_file = './data_sources/murphy_westerman/Murphy and Westerman.csv'
murphy_westerman_df = pd.read_csv(murphy_westerman_file)
murphy_westerman_df.head()

In [9]:
species_list+=murphy_westerman_df['Scientific name'].to_list()

In [ ]:
caves_fish_db_file = './data_sources/caves/caves_fish_db.csv'
caves_df = pd.read_csv(caves_fish_db_file)
caves_df.head()

In [11]:
species_list+=(caves_df['Genus'] + ' ' + caves_df['Species']).to_list()

In [ ]:
porter_file1 = './data_sources/megan_porter/megan_porter_extract_clean_2005.csv'
porter_df1 = pd.read_csv(porter_file1)
porter_df2 = pd.read_csv('./data_sources/megan_porter/porter_2006_table1_clean.tsv', sep='\t', index_col=0)
porter_df1.head()

In [ ]:
porter_df2.head()

In [14]:
species_list += (porter_df1['Full_Species'].to_list() + porter_df2['Taxon'].to_list())

In [ ]:
len(species_list)

In [ ]:
species_list = list(set(species_list))
len(species_list)

### <font color=#c994c7>In this case our dataframe has the full species name in one column so we can create a list directly and filter to create a list of all unique species names</font> 

In [17]:
#species_list = []

#for genus,species in zip(scp_df['Genus'],scp_df['Species']):
#    species_list.append(f'{genus} {species}')
#len(species_list)

In [18]:
#species_list = scp_df.index.to_list()
#len(species_list)

In [19]:
#unique_species_list = list(set(species_list)) 
#len(unique_species_list)

In [ ]:
ncbi_query_df = ncbi_fetch_opsins(email, job_label='mnm_on_all_dbs', out='mnm_on_all_dbs', species_list=species_list)

In [ ]:
ncbi_query_df.head()

In [ ]:
ncbi_query_df.shape

## <font color=#c994c7>Part 3 Objective</font> - Match sequence to it's closest MSP value based on OPTICS predictions

We'll need to...

- Query OPTICS with all the sequence data [bootstrap enabled]
- Extract Predictions
- Match to closest MSP value species-by-species [will need a list of the unique species names] // Match to MaxId as the foreign key

In [23]:
import os
import time 
import datetime
import warnings
import pandas as pd
from deepBreaks.preprocessing import read_data

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

In [24]:
report_dir = 'mnm_on_longcore_ncbi_data_2024-10-17' #re-define the report directory if needed
filtered_ncbi_query_file = f'./{report_dir}/filtered_ncbi_longcore_ops_query.csv'

In [25]:
# Example use for OPTICS
# python optics_predictions.py -in msp_mined_seqs.fasta -rd mined_msp_seqs -out mined_seq_predictions.tsv -m wildtype -e aa_prop -b True -ir msp_mined_seq_blastp_report.tsv -r bovine -s False -bsv msp_bs_viz.pdf
predFileData = f'./{report_dir}/optics_on_mined_longcore_seqs_2024-10-17_21-14-10/longcore_predictions.tsv'

In [26]:
scp_file = f'./{report_dir}/longcore_data/AnimalPhotopigmentsV1_1.csv'

## Clean-up post processing! 

- Upload mnm data to VPOD sheets
- If scp data came from a different database, also upload that to VPOD sheets

In [27]:
mnm_file = f"./mine_n_match_raw.tsv"